# Tunisair ETL — Project Overview

This notebook builds a simple, reproducible ETL pipeline for Tunisair flights using the Aviationstack API.

- Purpose: Extract flight records for airline IATA `TU`, then clean and export a ready‑to‑analyze dataset.
- Key steps:
  1) Extract: Call API and save raw JSON to auto‑incremented files (`raw_tunisair_flights.json`, `raw_tunisair_flights2.json`, ...).
  2) Combine: Merge all `raw_tunisair_flights*.json` into a single `raw_tunisair_flights_combined.json`.
  3) Dedupe: Remove exact duplicate records and save `raw_tunisair_flights_combined_dedup.json` (+ CSV).
  4) Transform: Normalize, clean, enrich, and summarize.
  5) Load: Export the final clean CSV `clean_tunisair_flights.csv`.


## Quickstart

- Requirements: Python 3.9+ and packages: `pandas`, `numpy`, `requests`.
- API key: Update the `API_KEY` value in the extract cell (Aviation Stack API Key).
- Run order:
  1) Libraries import
  2) Extract (auto‑incremented output file)
  3) 1.2 Combine Raw Files
  4) 1.3 Remove Duplicates
  5) Load Raw JSON Data → Transform → Load (CSV)

Outputs
- Raw: `raw_tunisair_flights*.json`
- Combined: `raw_tunisair_flights_combined.json`
- Deduped: `raw_tunisair_flights_combined_dedup.json`
- Clean: `clean_tunisair_flights.csv`



## 1. Extract Raw JSON Data

1.1. Importing required packages

In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")

Libraries imported successfully!
Pandas version: 2.3.3


Due to API limitations, this code extracts 100 flights. We will run it several times to ensure we get enough data for analyzing and visualizing.

In [2]:
API_KEY = "f9a3ebd329fea81d036463417e0b6ce9"

url = "http://api.aviationstack.com/v1/flights"
params = {
    "access_key": API_KEY,
    "airline_iata": "TU",
    "limit": 100
}

response = requests.get(url, params=params)

data = response.json()

# Save raw JSON data to the next available filename
import os
import re
import glob

def next_available_filename(base="raw_tunisair_flights", ext=".json"):
    files = glob.glob(f"{base}*{ext}")
    used = set()
    for fn in files:
        bn = os.path.basename(fn)
        if bn == f"{base}{ext}":
            used.add(1)
        else:
            m = re.match(rf"^{re.escape(base)}(\d+){re.escape(ext)}$", bn)
            if m:
                used.add(int(m.group(1)))
    if not used:
        return f"{base}{ext}"
    n = 2 if 1 in used else 1
    while n in used:
        n += 1
    return f"{base}{n}{ext}"

output_file = next_available_filename()
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"JSON data saved to {output_file}")
print("Records extracted:", len(data.get("data", [])))

JSON data saved to raw_tunisair_flights7.json
Records extracted: 100


### 1.2 Combine Raw Files
Combine all `raw_tunisair_flights*.json` files (only their `data` arrays) into a single file `raw_tunisair_flights_combined.json`. This ignores API `pagination` metadata.

In [3]:
# Combine all raw JSON files into one combined file
import json
import glob

input_files = sorted(glob.glob("raw_tunisair_flights*.json"))
combined_records = []
files_used = 0

for path in input_files:
    try:
        with open(path, "r", encoding="utf-8") as f:
            payload = json.load(f)
            records = payload.get("data", [])
            if records:
                combined_records.extend(records)
                files_used += 1
    except Exception as e:
        print(f"Skipping {path}: {e}")

combined_output = "raw_tunisair_flights_combined.json"
with open(combined_output, "w", encoding="utf-8") as f:
    json.dump({"data": combined_records}, f, indent=2, ensure_ascii=False)

print(f"Combined {files_used} files -> {combined_output}")
print(f"Total records combined: {len(combined_records)}")

Combined 9 files -> raw_tunisair_flights_combined.json
Total records combined: 2903


### 1.3 Remove Duplicates
Remove exact-duplicate flight records from `raw_tunisair_flights_combined.json` and save to `raw_tunisair_flights_combined_dedup.json` 

In [4]:
# Remove exact duplicates from the combined file and export JSON + CSV
import json
import pandas as pd

combined_input = "raw_tunisair_flights_combined.json"
dedup_output_json = "raw_tunisair_flights_combined_dedup.json"
dedup_output_csv = "raw_tunisair_flights_combined_dedup.csv"

with open(combined_input, "r", encoding="utf-8") as f:
    payload = json.load(f)

records = payload.get("data", [])

seen = set()
deduped = []
for rec in records:
    # Hash the entire record to identify exact duplicates
    key = json.dumps(rec, sort_keys=True, ensure_ascii=False)
    if key not in seen:
        seen.add(key)
        deduped.append(rec)

with open(dedup_output_json, "w", encoding="utf-8") as f:
    json.dump({"data": deduped}, f, indent=2, ensure_ascii=False)

print(f"Removed duplicates: {len(records) - len(deduped)}")
print(f"Deduped total records: {len(deduped)}")
print(f"Saved JSON -> {dedup_output_json}")



Removed duplicates: 2620
Deduped total records: 283
Saved JSON -> raw_tunisair_flights_combined_dedup.json


## 2. Load Raw JSON Data

In [5]:
# Load raw JSON data
with open('raw_tunisair_flights_combined_dedup.json', 'r') as f:
    raw_data = json.load(f)

print(f"\nTotal flights extracted: {len(raw_data['data'])}")


Total flights extracted: 283


In [6]:
# Explore the structure of one flight record
if raw_data['data']:
    sample_flight = raw_data['data'][0]
    print("Sample Flight Record Structure:")
    print(json.dumps(sample_flight, indent=2))

Sample Flight Record Structure:
{
  "flight_date": "2025-12-23",
  "flight_status": "scheduled",
  "departure": {
    "airport": "Carthage",
    "timezone": "Africa/Tunis",
    "iata": "TUN",
    "icao": "DTTA",
    "terminal": "M",
    "gate": null,
    "delay": null,
    "scheduled": "2025-12-23T20:05:00+00:00",
    "estimated": "2025-12-23T20:05:00+00:00",
    "actual": null,
    "estimated_runway": null,
    "actual_runway": null
  },
  "arrival": {
    "airport": "King Abdulaziz International",
    "timezone": "Asia/Riyadh",
    "iata": "JED",
    "icao": "OEJN",
    "terminal": "N",
    "gate": null,
    "baggage": null,
    "scheduled": "2025-12-24T02:30:00+00:00",
    "delay": null,
    "estimated": null,
    "actual": null,
    "estimated_runway": null,
    "actual_runway": null
  },
  "airline": {
    "name": "Tunisair",
    "iata": "TU",
    "icao": "TAR"
  },
  "flight": {
    "number": "913",
    "iata": "TU913",
    "icao": "TAR913",
    "codeshared": null
  },
  "aircraf

## 3. Transform - Normalize and Clean Data

In [7]:
# Normalize nested JSON into flat DataFrame
df_raw = pd.json_normalize(raw_data['data'])

print(f"Raw DataFrame Shape: {df_raw.shape}")
print(f"\nColumn Names ({len(df_raw.columns)} columns):")
print(df_raw.columns.tolist())

Raw DataFrame Shape: (283, 54)

Column Names (54 columns):
['flight_date', 'flight_status', 'aircraft', 'live', 'departure.airport', 'departure.timezone', 'departure.iata', 'departure.icao', 'departure.terminal', 'departure.gate', 'departure.delay', 'departure.scheduled', 'departure.estimated', 'departure.actual', 'departure.estimated_runway', 'departure.actual_runway', 'arrival.airport', 'arrival.timezone', 'arrival.iata', 'arrival.icao', 'arrival.terminal', 'arrival.gate', 'arrival.baggage', 'arrival.scheduled', 'arrival.delay', 'arrival.estimated', 'arrival.actual', 'arrival.estimated_runway', 'arrival.actual_runway', 'airline.name', 'airline.iata', 'airline.icao', 'flight.number', 'flight.iata', 'flight.icao', 'flight.codeshared', 'aircraft.registration', 'aircraft.iata', 'aircraft.icao', 'aircraft.icao24', 'flight.codeshared.airline_name', 'flight.codeshared.airline_iata', 'flight.codeshared.airline_icao', 'flight.codeshared.flight_number', 'flight.codeshared.flight_iata', 'flight

In [8]:
# Display first few rows
df_raw.head()

,flight_date,flight_status,aircraft,live,departure.airport,departure.timezone,departure.iata,departure.icao,departure.terminal,departure.gate,...,flight.codeshared.flight_iata,flight.codeshared.flight_icao,live.updated,live.latitude,live.longitude,live.altitude,live.direction,live.speed_horizontal,live.speed_vertical,live.is_ground
0,2025-12-23,scheduled,NaN,NaN,Carthage,Africa/Tunis,TUN,DTTA,M,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-12-23,scheduled,NaN,NaN,Orly,Europe/Paris,ORY,LFPO,4,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-12-23,scheduled,NaN,NaN,Carthage,Africa/Tunis,TUN,DTTA,M,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-12-23,scheduled,NaN,NaN,Heathrow,Europe/London,LHR,EGLL,4,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-12-23,scheduled,NaN,NaN,Orly,Europe/Paris,ORY,LFPO,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Check data types and missing values
print("Data Info:")
df_raw.info()

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 54 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   flight_date                      283 non-null    object 
 1   flight_status                    264 non-null    object 
 2   aircraft                         0 non-null      float64
 3   live                             0 non-null      float64
 4   departure.airport                283 non-null    object 
 5   departure.timezone               282 non-null    object 
 6   departure.iata                   283 non-null    object 
 7   departure.icao                   283 non-null    object 
 8   departure.terminal               227 non-null    object 
 9   departure.gate                   37 non-null     object 
 10  departure.delay                  111 non-null    float64
 11  departure.scheduled              283 non-null    object 
 12  departure.e

In [10]:
# Missing values analysis
missing_data = pd.DataFrame({
    'Column': df_raw.columns,
    'Missing_Count': df_raw.isnull().sum(),
    'Missing_Percentage': (df_raw.isnull().sum() / len(df_raw) * 100).round(2)
}).sort_values('Missing_Count', ascending=False)

print("\nMissing Values Analysis:")
print(missing_data[missing_data['Missing_Count'] > 0])


Missing Values Analysis:
                                                          Column  \
live                                                        live   
aircraft                                                aircraft   
flight.codeshared                              flight.codeshared   
arrival.gate                                        arrival.gate   
aircraft.registration                      aircraft.registration   
aircraft.iata                                      aircraft.iata   
live.is_ground                                    live.is_ground   
live.speed_vertical                          live.speed_vertical   
live.speed_horizontal                      live.speed_horizontal   
live.direction                                    live.direction   
aircraft.icao                                      aircraft.icao   
live.longitude                                    live.longitude   
live.latitude                                      live.latitude   
live.updated          

### Selecting relevant columns

In [11]:
# Select important columns for analysis
columns_to_keep = [
    'flight_date',
    'flight_status',
    'departure.airport',
    'departure.timezone',
    'departure.iata',
    'departure.delay',
    'departure.scheduled',
    'departure.estimated',
    'departure.actual',
    'arrival.airport',
    'arrival.timezone',
    'arrival.iata',
    'arrival.scheduled',
    'arrival.delay',
    'arrival.estimated',
    'arrival.actual',
    'airline.name',
    'airline.iata',
    'airline.icao',
    'flight.number',
    'flight.iata',
    'flight.icao',
]

# Keep only columns that exist in the dataframe
available_columns = [col for col in columns_to_keep if col in df_raw.columns]
df_clean = df_raw[available_columns].copy()

print(f"Selected {len(available_columns)} columns for cleaning")
print(f"Cleaned DataFrame Shape: {df_clean.shape}")

Selected 22 columns for cleaning
Cleaned DataFrame Shape: (283, 22)


## Renaming columns

In [12]:
# Rename columns with clearer names
column_mapping = {
    'flight_date':'flight_date',
    'flight_status':'flight_status',
    'departure.airport': 'departure_airport',
    'departure.timezone':'departure_timezone',
    'departure.iata':'departure_iata',
    'departure.delay':'departure_delay_minutes',
    'departure.scheduled':'departure_scheduled',
    'departure.estimated':'departure_estimated',
    'departure.actual':'departure_actual',
    'arrival.airport': 'arrival_airport',
    'arrival.timezone': 'arrival_timezone',
    'arrival.iata': 'arrival_iata',
    'arrival.scheduled': 'arrival_scheduled',
    'arrival.delay': 'arrival_delay_minutes',
    'arrival.estimated': 'arrival_estimated',
    'arrival.actual': 'arrival_actual',
    'airline.name': 'airline_name',
    'airline.iata': 'airline_iata',
    'airline.icao': 'airline_icao',
    'flight.number': 'flight_number',
    'flight.iata': 'flight_iata',
    'flight.icao': 'flight_icao',
}

df_clean.rename(columns=column_mapping, inplace=True)
print("Columns renamed successfully!")
print(f"\nNew column names:")
print(df_clean.columns.tolist())

Columns renamed successfully!

New column names:
['flight_date', 'flight_status', 'departure_airport', 'departure_timezone', 'departure_iata', 'departure_delay_minutes', 'departure_scheduled', 'departure_estimated', 'departure_actual', 'arrival_airport', 'arrival_timezone', 'arrival_iata', 'arrival_scheduled', 'arrival_delay_minutes', 'arrival_estimated', 'arrival_actual', 'airline_name', 'airline_iata', 'airline_icao', 'flight_number', 'flight_iata', 'flight_icao']


### 3.3 Convert Date/Time Columns

In [13]:
# Convert datetime columns
datetime_columns = [
    'departure_scheduled', 'departure_estimated', 'departure_actual',
    'arrival_scheduled', 'arrival_estimated', 'arrival_actual'
]

for col in datetime_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

print("Datetime columns converted successfully!")

Datetime columns converted successfully!


### 3.4 Clean and Convert Numeric Columns

In [14]:
# Convert numeric columns
numeric_columns = [
    'departure_delay_minutes', 'arrival_delay_minutes',
]

for col in numeric_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print("Numeric columns converted successfully!")

Numeric columns converted successfully!


### 3.5 Handle Missing Values

In [15]:
# Fill numeric delays with 0 if missing
if 'departure_delay_minutes' in df_clean.columns:
    df_clean['departure_delay_minutes'].fillna(0, inplace=True)

if 'arrival_delay_minutes' in df_clean.columns:
    df_clean['arrival_delay_minutes'].fillna(0, inplace=True)

# Fill categorical columns with "Unknown"
categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns
df_clean[categorical_cols] = df_clean[categorical_cols].fillna("Unknown")

print("Missing values handled!")
print(f"\nRemaining missing values:")
print(df_clean.isnull().sum().sum())

Missing values handled!

Remaining missing values:
483


### 3.6 Add Derived Columns

In [16]:
# Add helpful derived columns

# Extract date components from flight_date
if 'flight_date' in df_clean.columns:
    df_clean['flight_date'] = pd.to_datetime(df_clean['flight_date'])
    df_clean['year'] = df_clean['flight_date'].dt.year
    df_clean['month'] = df_clean['flight_date'].dt.month
    df_clean['day'] = df_clean['flight_date'].dt.day
    df_clean['day_of_week'] = df_clean['flight_date'].dt.day_name()

# Calculate total delay (if both departure and arrival delays exist)
if 'departure_delay_minutes' in df_clean.columns and 'arrival_delay_minutes' in df_clean.columns:
    df_clean['total_delay_minutes'] = df_clean['departure_delay_minutes'] + df_clean['arrival_delay_minutes']

# Create a route identifier
if 'departure_iata' in df_clean.columns and 'arrival_iata' in df_clean.columns:
    df_clean['route'] = df_clean['departure_iata'].astype(str) + ' → ' + df_clean['arrival_iata'].astype(str)

# Flight is delayed flag
if 'total_delay_minutes' in df_clean.columns:
    df_clean['is_delayed'] = df_clean['total_delay_minutes'] > 15

print("Derived columns added successfully!")
print(f"\nFinal DataFrame Shape: {df_clean.shape}")

Derived columns added successfully!

Final DataFrame Shape: (283, 29)


## 4. Data Quality Summary

In [17]:
# Display cleaned data summary
print("="*60)
print("CLEANED DATA SUMMARY")
print("="*60)
print(f"\nTotal Records: {len(df_clean)}")
print(f"Total Columns: {len(df_clean.columns)}")
print(f"\nMemory Usage: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nData Types:")
print(df_clean.dtypes.value_counts())

CLEANED DATA SUMMARY

Total Records: 283
Total Columns: 29

Memory Usage: 0.26 MB

Data Types:
object                 15
datetime64[ns, UTC]     6
int32                   3
float64                 3
datetime64[ns]          1
bool                    1
Name: count, dtype: int64


In [18]:
# Display statistical summary for numeric columns
print("\nNumeric Columns Summary:")
df_clean.describe()


Numeric Columns Summary:


,flight_date,departure_delay_minutes,arrival_delay_minutes,year,month,day,total_delay_minutes
count,283,283.000000,283.000000,283.0,283.0,283.000000,283.000000
mean,2025-12-22 08:54:16.537102592,23.862191,37.833922,2025.0,12.0,22.371025,61.696113
min,2025-12-21 00:00:00,0.000000,0.000000,2025.0,12.0,21.000000,0.000000
25%,2025-12-22 00:00:00,0.000000,0.000000,2025.0,12.0,22.000000,0.000000
50%,2025-12-22 00:00:00,0.000000,12.000000,2025.0,12.0,22.000000,16.000000
75%,2025-12-23 00:00:00,35.500000,64.000000,2025.0,12.0,23.000000,94.000000
max,2025-12-24 00:00:00,250.000000,249.000000,2025.0,12.0,24.000000,499.000000
std,NaN,42.086437,48.900507,0.0,0.0,0.753343,83.901662


In [19]:
# Flight status distribution
if 'flight_status' in df_clean.columns:
    print("\nFlight Status Distribution:")
    print(df_clean['flight_status'].value_counts())


Flight Status Distribution:
flight_status
scheduled    120
active        77
landed        67
Unknown       19
Name: count, dtype: int64


In [20]:
# Display sample of cleaned data
print("\nSample of Cleaned Data:")
df_clean.head(10)


Sample of Cleaned Data:


,flight_date,flight_status,departure_airport,departure_timezone,departure_iata,departure_delay_minutes,departure_scheduled,departure_estimated,departure_actual,arrival_airport,...,flight_number,flight_iata,flight_icao,year,month,day,day_of_week,total_delay_minutes,route,is_delayed
0,2025-12-23,scheduled,Carthage,Africa/Tunis,TUN,0.0,2025-12-23 20:05:00+00:00,2025-12-23 20:05:00+00:00,NaT,King Abdulaziz International,...,913,TU913,TAR913,2025,12,23,Tuesday,0.0,TUN → JED,False
1,2025-12-23,scheduled,Orly,Europe/Paris,ORY,0.0,2025-12-23 19:15:00+00:00,2025-12-23 19:15:00+00:00,NaT,Djerba-Zarzis,...,635,TU635,TAR635,2025,12,23,Tuesday,0.0,ORY → DJE,False
2,2025-12-23,scheduled,Carthage,Africa/Tunis,TUN,0.0,2025-12-23 18:25:00+00:00,2025-12-23 18:25:00+00:00,NaT,King Abdulaziz International,...,713,TU713,TAR713,2025,12,23,Tuesday,0.0,TUN → JED,False
3,2025-12-23,scheduled,Heathrow,Europe/London,LHR,0.0,2025-12-23 16:50:00+00:00,2025-12-23 16:50:00+00:00,NaT,Carthage,...,791,TU791,TAR791,2025,12,23,Tuesday,0.0,LHR → TUN,False
4,2025-12-23,scheduled,Orly,Europe/Paris,ORY,0.0,2025-12-23 13:00:00+00:00,2025-12-23 13:00:00+00:00,NaT,Carthage,...,2725,TU2725,TAR2725,2025,12,23,Tuesday,0.0,ORY → TUN,False
5,2025-12-23,scheduled,Carthage,Africa/Tunis,TUN,0.0,2025-12-23 17:00:00+00:00,2025-12-23 17:00:00+00:00,NaT,Blaise Diagne International Airport\r\n,...,343,TU343,TAR343,2025,12,23,Tuesday,0.0,TUN → DSS,False
6,2025-12-23,scheduled,Carthage,Africa/Tunis,TUN,0.0,2025-12-23 16:55:00+00:00,2025-12-23 16:55:00+00:00,NaT,Orly,...,724,TU724,TAR724,2025,12,23,Tuesday,10.0,TUN → ORY,False
7,2025-12-23,scheduled,MÃ©rignac,Europe/Paris,BOD,0.0,2025-12-23 16:35:00+00:00,2025-12-23 16:35:00+00:00,NaT,Carthage,...,629,TU629,TAR629,2025,12,23,Tuesday,0.0,BOD → TUN,False
8,2025-12-23,scheduled,Orly,Europe/Paris,ORY,0.0,2025-12-23 17:00:00+00:00,2025-12-23 17:00:00+00:00,NaT,Carthage,...,723,TU723,TAR723,2025,12,23,Tuesday,0.0,ORY → TUN,False
9,2025-12-23,scheduled,Es Senia,Africa/Algiers,ORN,0.0,2025-12-23 15:40:00+00:00,2025-12-23 15:40:00+00:00,NaT,Carthage,...,709,TU709,TAR709,2025,12,23,Tuesday,0.0,ORN → TUN,False


## 5. Load - Export Clean Dataset

In [21]:
# Export to CSV
output_file = 'clean_tunisair_flights.csv'
df_clean.to_csv(output_file, index=False, encoding='utf-8')

print(f"✅ Clean dataset exported to: {output_file}")
print(f"Records exported: {len(df_clean)}")
print(f"Columns exported: {len(df_clean.columns)}")

✅ Clean dataset exported to: clean_tunisair_flights.csv
Records exported: 283
Columns exported: 29
